In [10]:
import pandas as pd
import numpy as np
import os
import json
import re

all the imports. ** explinations in this project r on the bottom of the code **

In [11]:
data = pd.read_excel('routine.xlsx', sheet_name=2 ,skiprows=4)
data.columns = data.columns.str.replace('\n', '_')
data.columns = data.columns.str.replace(':', '')
data.columns = data.columns.str.replace(' ', '_')
data.columns = data.columns.str.replace(r'\.\d+', '', regex=True)
dataframe = data.fillna('-') 
dataframe.columns = dataframe.columns.str.upper()

skips the unnecessary sheets and also skipps the first 4 rows which r the headings of the sheet.

In [12]:
data_cleaning = dataframe.drop(["CLASS_SIZE", "ASSIGNED_HALL", "HALL_CAPACITY_MATCH", "UNNAMED_57"], axis='columns')
# ? UNNAMED_57 => is break
# len(data.columns)
indexList = list(range(61))
for i in range(2, 61, 5):
    indexList.remove(i)
clean_data = data_cleaning.iloc[:, indexList]

adds the day in all the rows of the day column 

In [13]:
def fill_days(column):
    current_day = None
    new_column = column.copy()  # Create a copy to avoid warnings
    for i, value in enumerate(new_column):
        if pd.notna(value) and value != '-':
            current_day = value
        elif value == '-' and current_day is not None:
            new_column.iloc[i] = current_day
    return new_column

# Make sure clean_data is a copy, not a view
clean_data = clean_data.copy()

# Apply the function to the 'DAY' column
clean_data.loc[:, 'DAY'] = fill_days(clean_data['DAY'])

this fills in all the columns in day secction. all the columns r filled wit day name

In [14]:
days_of_week = ['MONDAY', 'TUESDAY', 'WEDNESDAY', 'THURSDAY', 'FRIDAY', 'SATURDAY']
monday_df, tuesday_df, wednesday_df, thursday_df, friday_df, saturday_df = [clean_data.loc[clean_data['DAY'] == day] for day in days_of_week]

# Create a new folder to store the CSV files
new_folder = "excel data"
if not os.path.exists(new_folder):
    os.makedirs(new_folder)

for day in days_of_week:
    day_data = clean_data[clean_data['DAY'] == day]
    filename = f"{day}.xlsx"  # Create a unique filename for each day
    file_path = os.path.join(new_folder, filename)  # Join the folder path with the filename
    day_data.to_excel(file_path, index=False)


makes indivigual dataframes for every 'day' section of the week

In [15]:

# ! Define the time intervals to group data
time_intervals = ['7.00 - 7.45', '7.45 - 8.30', '8.30 - 9.20', '9.20 - 10.10', '10.10 - 11.00', '11.00 - 11.50', '11.50 - 12.35', '1.00 - 1.45', '1.45 - 2.30', '2.30 - 3.15', '3.15 - 4.00', '4.00 - 4.45']

# Extract columns except 'DAY'
columns_except_day = clean_data.columns[1:]  # Skip 'DAY' column

# Group every 4 columns together and create multi-level column names
n_sets = len(columns_except_day) // 4
new_columns = [('DAY', '')]  # Keep 'DAY' at the start

for i in range(n_sets):
    time_range = time_intervals[i]  # Assign each group a time interval
    new_columns.extend([(time_range, col) for col in ['COURSE_NAME', 'COURSE_CODE_SECTION_&_SHIFT', 'FACULTY_INITIALS', 'HALL']])

# List of DataFrames to update
dataframes = [monday_df, tuesday_df, wednesday_df, thursday_df, friday_df, saturday_df]


# Assign the new MultiIndex to each DataFrame's columns
for df in dataframes:
    df.columns = pd.MultiIndex.from_tuples(new_columns)
    


adds time intervals as headers of the original columns. adds time on top of the columns

In [16]:
def convert_to_24_hour(time_str):
    # Convert time from 12-hour to 24-hour format for sorting
    match = re.match(r'(\d{1,2})\.(\d{2})', time_str)
    if match:
        hour, minute = int(match.group(1)), int(match.group(2))
        # Assuming all times are in AM before 12:00 and PM after 12:00
        if hour < 7:  # Assuming the day starts at 7:00 AM, anything below is PM
            hour += 12
        return hour + minute / 60.0  # Convert to float for sorting
    return float('inf')  # Fallback for unrecognized formats

def sort_time_intervals(time_intervals):
    # Sort the intervals based on their start time in 24-hour format
    return sorted(time_intervals, key=lambda x: convert_to_24_hour(x.split(' - ')[0]))

In [17]:
def convert_df_to_json(df, day):
    # Initialize the dictionary to hold the structured data
    structured_json = {}
   
    # Extract time intervals from the MultiIndex
    time_intervals = df.columns.levels[0]
   
    # Sort the time intervals numerically
    time_intervals = sort_time_intervals(time_intervals)
   
    # Iterate over each time interval
    for time_interval in time_intervals:
        if time_interval in df.columns.levels[0]:
            # Filter DataFrame to get the relevant columns for the current time interval
            interval_df = df[time_interval]
           
            # Ensure interval_df is a DataFrame
            if isinstance(interval_df, pd.Series):
                interval_df = interval_df.to_frame()
           
            # Convert the DataFrame to a list of dictionaries
            records = interval_df.reset_index(drop=True).to_dict(orient='records')
           
            # Add the data to the structured JSON
            structured_json[time_interval] = records
        else:
            # Ensure that even empty time intervals are included in the JSON
            structured_json[time_interval] = []
    
    # Return the final structured JSON with the day as the key
    return {day: structured_json}

In [18]:
# List of days
days = ['monday', 'tuesday', 'wednesday', 'thursday', 'friday', 'saturday']

# Create a new folder to store the JSON files
new_folder = "json data"
os.makedirs(new_folder, exist_ok=True)

# Process each day
for day in days:
    try:
        # Assume the DataFrame for each day is named '{day}_df'
        df = globals()[f'{day}_df']

        # Convert DataFrame to JSON format
        day_json = convert_df_to_json(df, day)

        # Save it to a .json file within the new folder
        filename = os.path.join(new_folder, f'{day}_data.json')
        with open(filename, 'w') as f:
            json.dump(day_json, f, indent=4)

        print(f"JSON data saved to {filename}")
    except Exception as e:
        print(f"An error occurred processing {day}: {e}")
print("...")
print("All data has been processed and saved!")

JSON data saved to json data\monday_data.json
JSON data saved to json data\tuesday_data.json
JSON data saved to json data\wednesday_data.json
JSON data saved to json data\thursday_data.json
JSON data saved to json data\friday_data.json
JSON data saved to json data\saturday_data.json
...
All data has been processed and saved!
